In [1]:
import pandas as pd
import numpy as np

In [ ]:
# read data


In [ ]:
# filter to 2019

df['INTERVIEW_DATE'] = pd.to_datetime(df['INTERVIEW_DATE'])
df = df[df['INTERVIEW_DATE'].dt.year == 2019]
df = df.reset_index(drop = True)

In [ ]:
# date of departure = interview date
# month of arrival = month(date of departure - length of stay)

df = df.rename(columns = {'INTERVIEW_DATE': 'DATE_OF_DEPARTURE'})
df['DATE_OF_ARRIVAL'] = df['DATE_OF_DEPARTURE'] - pd.to_timedelta(df['B10'], unit = 'd')
df['MONTH_OF_ARRIVAL'] = df['DATE_OF_ARRIVAL'].dt.month

In [ ]:
# mode of transport (depature/arrival) = air/land/sea

mode_map = {}
df['MODE_OF_DEPARTURE'] = df['M1'].map(mode_map)
df['MODE_OF_ARRIVAL'] = df['B2'].map(mode_map)

In [ ]:
# language of interview

df = df.rename(columns = {'LanguageQ': 'LANGUAGE_OF_INTERVIEW'})

In [ ]:
# type of passenger

df = df.rename(columns = {'A1': 'PASSENGER_TYPE'})

In [ ]:
# age band

age_map = {}
df['AGE_BAND'] = df['a2.r'].map(age_map)

In [ ]:
# nationality

df = df.rename(columns = {'A4': 'NATIONALITY'})

In [ ]:
# country of residence

df = df.rename(columns = {'A5': 'COUNTRY_OF_RESIDENCE'})

In [ ]:
# time since last visit

time_map = {}
df['TIME_SINCE_LAST_VISIT'] = df['B1b'].map(time_map).fillna(0)

In [ ]:
# place of origin =
# - state in Malaysia if mode of arrival == land
# - country before Singapore o/w
# - country of residence if missing
# - nationality if missing

df['PLACE_OF_ORIGIN'] = np.where(df['MODE_OF_ARRIVAL'] == 'Land', df['B4'], df['B7a'])
df['PLACE_OF_ORIGIN'] = df['PLACE_OF_ORIGIN'].fillna(df['COUNTRY_OF_RESIDENCE']).fillna(df['NATIONALITY'])

In [ ]:
# destination =
# - country after Singapore
# - country of residence if missing
# - nationality if missing

df['DESTINATION'] = df['B9a'].fillna(df['COUNTRY_OF_RESIDENCE']).fillna(df['NATIONALITY'])

In [ ]:
# mode of transport = passenger class x mode of departure



In [ ]:
# nights spent before/after Singapore

df = df.rename(columns = {'B7c': 'NIGHTS_BEFORE_SG',
                          'B9c': 'NIGHTS_AFTER_SG'})

# impute nights spent before Singapore with
# - 0 if place of origin == country of residence or nationality
# - mode when grouped by same place of origin, if data is available
# - mode of all filled values o/w

conditions = (df['PLACE_OF_ORIGIN'] == df['COUNTRY_OF_RESIDENCE']) | (df['PLACE_OF_ORIGIN'] == df['NATIONALITY'])
df.loc[conditions, 'NIGHTS_BEFORE_SG'] = df.loc[conditions, 'NIGHTS_BEFORE_SG'].fillna(0)
modes = df.groupby('PLACE_OF_ORIGIN')['NIGHTS_BEFORE_SG'].apply(lambda x: x.mode().iloc[0])
df['NIGHTS_BEFORE_SG'] = df.apply(
    lambda row: modes[row['PLACE_OF_ORIGIN']] if pd.isna(row['NIGHTS_BEFORE_SG']) else row['NIGHTS_BEFORE_SG'],
    axis = 1
)
overall_mode = df['NIGHTS_BEFORE_SG'].mode().iloc[0]
df['NIGHTS_BEFORE_SG'].fillna(overall_mode, inplace = True)

# impute nights spent after Singapore with
# - 0 if destination == country of residence or nationality
# - mode when grouped by same destination, if data is available
# - mode of all filled values o/w

conditions = (df['DESTINATION'] == df['COUNTRY_OF_RESIDENCE']) | (df['PLACE_OF_ORIGIN'] == df['NATIONALITY'])
df.loc[conditions, 'NIGHTS_AFTER_SG'] = df.loc[conditions, 'NIGHTS_AFTER_SG'].fillna(0)
modes = df.groupby('DESTINATION')['NIGHTS_AFTER_SG'].apply(lambda x: x.mode().iloc[0])
df['NIGHTS_AFTER_SG'] = df.apply(
    lambda row: modes[row['DESTINATION']] if pd.isna(row['NIGHTS_AFTER_SG']) else row['NIGHTS_AFTER_SG'],
    axis = 1
)
overall_mode = df['NIGHTS_AFTER_SG'].mode().iloc[0]
df['NIGHTS_AFTER_SG'].fillna(overall_mode, inplace = True)

In [ ]:
# length of stay

df = df.rename(columns = {'B10': 'LENGTH_OF_STAY'})

In [ ]:
# travel companion(s)

# includes spouse
df['TRAVEL_COMPANION=SPOUSE'] = df['B12_2'].fillna(0)

# includes partner
df['TRAVEL_COMPANION=PARTNER'] = df['B12_3'].fillna(0)

# number of parents
df['N_PARENTS'] = df['B12NUMBER_2'].fillna(0)

# number of grandparents
df['N_GRANDPARENTS'] = df['B12NUMBER_3'].fillna(0)

# number of siblings
df['N_SIBLINGS'] = df['B12NUMBER_4'].fillna(0)

# number of other relatives
df['N_OTHER_RELATIVES'] = df['B12NUMBER_5'].fillna(0)

# number of friends
df['N_FRIENDS'] = df['B12NUMBER_6'].fillna(0)

# number of other companions
df['N_OTHERS'] = df[['B12NUMBER_7', 'B12NUMBER_8', 'B12NUMBER_9']].sum(axis=1, skipna=True).fillna(0)

# number of children
df['N_CHILDREN'] = df['B12NUMBER_2'].fillna(0)

# boolean for child(ren) in 0-12, 13-19 age band
df['BOOL_CHILD_0to12Years'] = df['B12b_1'].fillna(0)
df['BOOL_CHILD_13to19Years'] = df['B12b_2'].fillna(0)

In [ ]:
# main purpose of visit

pov_map = {
    'B13a_111': 'Event',
    'B13a_112': 'Leisure',
    'B13a_113': 'Accompany',
    'B13a_114': 'Visit',
    'B13a_115': 'Gather Info',
    'B13a_116': 'Business',
    'B13a_117': 'Healthcare',
    'B13a_118': 'Education',
    'B13a_119': 'Other'
}

def MainPOV(row):
    active_columns = [col for col in columns if row[col] == 1]
    if len(active_columns) == 1:
        return pov_map[active_columns[0]]
    else:
        return 'Other'
    
df['MAIN_POV'] = df.apply(MainPOV, axis=1)

In [ ]:
# map purpose of visit subcategory

subpov_map = {
    'B13b_1': 'POV = (Event) Family Entertainment',
    'B13b_2': 'POV = (Event) Performing Arts',
    'B13b_3': 'POV = (Event) Sporting',
    'B13b_4': 'POV = (Event) Music-related',
    'B13b_5': 'POV = (Event) Cultural Festival',
    'B13b_6': 'POV = (Event) Party',
    'B13b_7': 'POV = (Event) Other',
    'B13b_8': 'POV = (Leisure) Food',
    'B13b_9': 'POV = (Leisure) Nightlife',
    'B13b_10': 'POV = (Leisure) Shopping',
    'B13b_11': 'POV = (Leisure) Experience Cultures',
    'B13b_12': 'POV = (Leisure) Cruise',
    'B13b_13': 'POV = (Leisure) Sightseeing',
    'B13b_14': 'POV = (Leisure) Rest & Relax',
    'B13b_15': 'POV = (Leisure) Integrated Resorts',
    'B13b_771': 'POV = (Leisure) Other',
    'B13b_21': 'POV = (Accompany) Business Visitor',
    'B13b_22': 'POV = (Accompany) Outpatient Consultation / Treatment',
    'B13b_23': 'POV = (Accompany) Day Surgery',
    'B13b_24': 'POV = (Accompany) Inpatient Consultation / Treatment',
    'B13b_25': 'POV = (Accompany) International Student',
    'B13b_772': 'POV = (Accompany) Other',
    'B13b_31': 'POV = (Visit) Locals',
    'B13b_32': 'POV = (Visit) International Student',
    'B13b_33': 'POV = (Gather Info) Healthcare',
    'B13b_34': 'POV = (Gather Info) Education',
    'B13b_773': 'POV = (Gather Info) Other',
    'B13b_41': 'POV = (Business) Meeting',
    'B13b_42': 'POV = (Business) Sponsored Holiday',
    'B13b_43': 'POV = (Business) Convention / Conference',
    'B13b_44': 'POV = (Business) Exhibition / Trade Show',
    'B13b_45': 'POV = (Business) General',
    'B13b_774': 'POV = (Business) Other',
    'B13b_51': 'POV = (Healthcare) Outpatient Consultation / Treatment',
    'B13b_52': 'POV = (Healthcare) Day Surgery',
    'B13b_53': 'POV = (Healthcare) Inpatient Consultation / Treatment',
    'B13b_775': 'POV = (Healthcare) Other',
    'B13b_61': 'POV = (Education) Company Training',
    'B13b_62': 'POV = (Education) Executive Training',
    'B13b_63': 'POV = (Education) Study Mission',
    'B13b_64': 'POV = (Education) Skills Development',
    'B13b_65': 'POV = (Education) Personal Enrichment',
    'B13b_66': 'POV = (Education) School Trip',
    'B13b_67': 'POV = (Education) Student Event',
    'B13b_68': 'POV = (Education) Student Enrichment',
    'B13b_776': 'POV = (Education) Other',
    'B13b_120': 'POV = (Other) Singapore as a Non-overnight Layover',
    'B13b_121': 'POV = (Other) Singapore as an Overnight Layover',
    'B13b_122': 'POV = (Other) Job Opportunities',
    'B13b_124': 'POV = (Other) Visa / Permit-related',
    'B13b_996': 'POV = (Other) Other'
}
df.rename(columns = subpov_map, inplace = True)
df[list(subpov_map.values)] = df[list(subpov_map.values)].applymap(lambda x: False if pd.isna(x) or x == 0 else True)

In [ ]:
# purchased travel package

df['TRAVEL_PACKAGE'] = df['C1a'].fillna(0)
df['tr.type'].fillna("")

# package type = travel type x inclusion of travel fare

def PackageType(row):
    tr_type = row['tr.type']
    if row['tr.type']:
        
    if row['C2']:
        fare_type = " (incl. Travel Fare)"
    else:
        fare_type = ""
    return f"{tr_type}{fare_type}"

df['PACKAGE_TYPE'] = df.apply(PackageType, axis=1)
df['PACKAGE_TYPE'] = df['PACKAGE_TYPE'].astype(str)

# cost of travel package

df['$_PACKAGE_PER_PAX'] = df['C1b.r'].fillna(0)

In [ ]:
# accommodation
# - if no nights in package, fill with 0
# - if no nights in paid accommodation, fill with nights in package
# - if no nights in unpaid accommodation, fill with length of stay - nights in paid accommodation
# - if no specified pax, impute with 1 + total number of travel companions

df['NIGHTS_IN_PACKAGE'] = df['C1c'].fillna(0)
df['NIGHTS_IN_PAID_ACCOMMODATION'] = df['C4c_8'].fillna(df['NIGHTS_IN_PACKAGE'], inplace=True)
df['NIGHTS_IN_UNPAID_ACCOMMODATION'] = df['C4c_12'].fillna(df['LENGTH_OF_STAY'] - df['NIGHTS_IN_PAID_ACCOMMODATION'], inplace=True)

df['$_ACCOMMODATION_TOTAL'] = df[['c4d_1.r', 'c4d_2.r', 'c4d_3.r', 'c4d_4.r', 'c4d_5.r', 'c4d_6.r', 'c4d_7.r', 'c4d_8.r']].sum(axis=1)

total_companions = [
    'TRAVEL_COMPANION=SPOUSE',
    'TRAVEL_COMPANION=PARTNER',
    'N_PARENTS',
    'N_GRANDPARENTS',
    'N_SIBLINGS',
    'N_OTHER_RELATIVES',
    'N_FRIENDS',
    'N_OTHERS',
    'N_CHILDREN'
]
n_pax = df[total_companions].sum(axis=1) + 1
df['PAX_IN_ACCOMMODATION'] = df['C5'].fillna(n_pax, inplace=True)

In [ ]:
# food & beverages
# - not in package unless declared
# - if no total spend, impute with sum of spend across all dining options
# - if no specified pax, impute with pax in accommodation

df['F&B_IN_PACKAGE'] = df['C6_1'].fillna(0)
df['F&B_HAWKER_IN_PACKAGE'] = df['C6b_1_1'].fillna(0)
df['F&B_CASUAL_IN_PACKAGE'] = df['C6b_2_1'].fillna(0)
df['F&B_FINE_IN_PACKAGE'] = df['C6b_3_1'].fillna(0)
df['F&B_OTHER_IN_PACKAGE'] = df['C6b_4_1'].fillna(0)

dining_map = {
    'c6c_1.r': '$_F&B_HAWKER',
    'c6c_2.r': '$_F&B_CASUAL',
    'c6c_3.r': '$_F&B_FINE',
    'c6c_4.r': '$_F&B_OTHER'
}
df.rename(columns = dining_map, inplace = True)
df['$_F&B_TOTAL'] = df['c6.tot'].fillna(df[['$_F&B_HAWKER','$_F&B_CASUAL','$_F&B_FINE','$_F&B_OTHER']].sum(axis=1))

df['PAX_DINING'] = df['C6d'].fillna(df['PAX_IN_ACCOMMODATION'])

In [ ]:
# transportation
# - not in package unless declared
# - if no total spend, impute with sum of spend in vehicle and other
# - if no specified pax, impute with pax in accommodation

df['TRANSPORT_IN_PACKAGE'] = df['C7a_1'].fillna(0)

transport_map = {
    'C7b_1.r': '$_TRANSPORT_VEHICLE',
    'C7b_2.r': '$_TRANSPORT_OTHER',
}
df.rename(columns = transport_map, inplace = True)
df['$_TRANSPORT_TOTAL'] = df['c7.tot'].fillna(df[['$_TRANSPORT_VEHICLE','$_TRANSPORT_OTHER']].sum(axis=1))

df['PAX_TRANSPORT'] = df['C7c'].fillna(df['PAX_IN_ACCOMMODATION'])

In [ ]:
# tour purchase method

method_map = {
    'C8ii_1': 'Pre-arrival Online',
    'C8ii_2': 'Post-arrival Online',
    'C8ii_4': 'Tour Desk',
    'C8ii_5': 'Hotel',
    'C8ii_6': 'Visitor Centre'
}

def PurchaseMethod(row):
    active_columns = [col for col in columns if row[col] == 1]
    if len(active_columns) == 1:
        return method_map[active_columns[0]]
    else:
        return 'Other'
    
df['TOUR_PURCHASE_METHOD'] = df.apply(PurchaseMethod, axis=1)

In [ ]:
# friends/family in Singapore
# - no unless declared

df['FRIENDS_FAMILY_IN_SG'] = df['C9'].fillna(0)

In [ ]:
# sightseeing & entertainment

